# Tokenizer for Indie Language


In [1]:
#Initializing the data for vocabulary
corpus2 = [
    "প্রেমিকের সঙ্গে বান্ধবীর ঘনিষ্ঠতার ভিডিয়ো তুলে টাকা আদায়! কলকাতার শিক্ষিকা-সহ গ্রেফতার দুই",
    "পুলিশের একটি সূত্র জানাচ্ছে, ধৃত স্কুলশিক্ষিকা পঞ্চসায়রের একটি বেসরকারি স্কুলে পড়ান। তাঁর সঙ্গে প্রেমের সম্পর্ক রয়েছে কলকাতা মেট্রোয় কর্মরত এক যুবকের। দু’জনকেই গ্রেফতার করা হয়েছে।"
]

**Defining the Pre-Tokenizer**

In [2]:
from collections import defaultdict
def tokenizer(corpus2):
  word_counts = defaultdict(int)
  for sentence in corpus2:
      words = sentence.split()
      for word in words:
          word_counts[word] += 1
  return word_counts



In [3]:
#Splitting the sentence into words
word_counts = tokenizer(corpus2)

In [4]:
print(word_counts)

defaultdict(<class 'int'>, {'প্রেমিকের': 1, 'সঙ্গে': 2, 'বান্ধবীর': 1, 'ঘনিষ্ঠতার': 1, 'ভিডিয়ো': 1, 'তুলে': 1, 'টাকা': 1, 'আদায়!': 1, 'কলকাতার': 1, 'শিক্ষিকা-সহ': 1, 'গ্রেফতার': 2, 'দুই': 1, 'পুলিশের': 1, 'একটি': 2, 'সূত্র': 1, 'জানাচ্ছে,': 1, 'ধৃত': 1, 'স্কুলশিক্ষিকা': 1, 'পঞ্চসায়রের': 1, 'বেসরকারি': 1, 'স্কুলে': 1, 'পড়ান।': 1, 'তাঁর': 1, 'প্রেমের': 1, 'সম্পর্ক': 1, 'রয়েছে': 1, 'কলকাতা': 1, 'মেট্রোয়': 1, 'কর্মরত': 1, 'এক': 1, 'যুবকের।': 1, 'দু’জনকেই': 1, 'করা': 1, 'হয়েছে।': 1})


In [5]:
vocab = []
#compute the base vocabulary, formed by all the characters used in the corpus:
for word in word_counts.keys():
    for letter in word:
        if letter not in vocab:
            vocab.append(letter)
vocab.sort()

print(vocab)

['!', ',', '-', '।', 'ঁ', 'আ', 'ই', 'এ', 'ক', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঞ', 'ট', 'ঠ', 'ড', 'ত', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ো', '্', 'য়', '’']


**Split each word into individual characters, to be able to start training**

In [6]:
splits = {word: [c for c in word] for word in word_counts.keys()}

In [7]:
# A function that computes the frequency of each pair
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_counts.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [8]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('প', '্'): 2
('্', 'র'): 6
('র', 'ে'): 5
('ে', 'ম'): 2
('ম', 'ি'): 1
('ি', 'ক'): 5


In [9]:
#finding the most frequent pair
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('্', 'র') 6


In [10]:
 #applying the merge in our splits dictionary
def merge_pair(a, b, splits):
    for word in word_counts:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

# Training

In [11]:
vocab_size = 50
merges= {}
vocab = []

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [12]:
print(merges)

{('্', 'র'): '্র', ('ত', 'া'): 'তা', ('ক', 'া'): 'কা', ('ে', 'র'): 'ের', ('্র', 'ে'): '্রে', ('তা', 'র'): 'তার', ('ু', 'ল'): 'ুল', ('ি', 'ক'): 'িক', ('া', 'ন'): 'ান', ('এ', 'ক'): 'এক', ('ছ', 'ে'): 'ছে', ('্', 'ক'): '্ক', ('প', '্রে'): 'প্রে', ('প্রে', 'ম'): 'প্রেম', ('স', 'ঙ'): 'সঙ', ('সঙ', '্'): 'সঙ্', ('সঙ্', 'গ'): 'সঙ্গ', ('সঙ্গ', 'ে'): 'সঙ্গে', ('ুল', 'ে'): 'ুলে', ('া', 'য়'): 'ায়', ('ক', 'ল'): 'কল', ('কল', 'কা'): 'কলকা', ('শ', 'িক'): 'শিক', ('শিক', '্'): 'শিক্', ('শিক্', 'ষ'): 'শিক্ষ', ('শিক্ষ', 'ি'): 'শিক্ষি', ('শিক্ষি', 'কা'): 'শিক্ষিকা', ('গ', '্রে'): 'গ্রে', ('গ্রে', 'ফ'): 'গ্রেফ', ('গ্রেফ', 'তার'): 'গ্রেফতার', ('দ', 'ু'): 'দু', ('এক', 'ট'): 'একট', ('একট', 'ি'): 'একটি', ('স', '্ক'): 'স্ক', ('য়', 'ে'): 'য়ে', ('য়ে', 'ছে'): 'য়েছে', ('ক', 'র'): 'কর', ('প্রেম', 'িক'): 'প্রেমিক', ('প্রেমিক', 'ের'): 'প্রেমিকের', ('ব', 'ান'): 'বান', ('বান', '্'): 'বান্', ('বান্', 'ধ'): 'বান্ধ', ('বান্ধ', 'ব'): 'বান্ধব', ('বান্ধব', 'ী'): 'বান্ধবী', ('বান্ধবী', 'র'): 'বান্ধবীর', ('ঘ', 'ন'): 'ঘন', ('ঘন', 

In [13]:
print(vocab)

['্র', 'তা', 'কা', 'ের', '্রে', 'তার', 'ুল', 'িক', 'ান', 'এক', 'ছে', '্ক', 'প্রে', 'প্রেম', 'সঙ', 'সঙ্', 'সঙ্গ', 'সঙ্গে', 'ুলে', 'ায়', 'কল', 'কলকা', 'শিক', 'শিক্', 'শিক্ষ', 'শিক্ষি', 'শিক্ষিকা', 'গ্রে', 'গ্রেফ', 'গ্রেফতার', 'দু', 'একট', 'একটি', 'স্ক', 'য়ে', 'য়েছে', 'কর', 'প্রেমিক', 'প্রেমিকের', 'বান', 'বান্', 'বান্ধ', 'বান্ধব', 'বান্ধবী', 'বান্ধবীর', 'ঘন', 'ঘনি', 'ঘনিষ', 'ঘনিষ্', 'ঘনিষ্ঠ']


In [19]:
def tokenize(text):
    pre_tokenized_text = tokenizer(text)

    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:

                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split
    return sum(splits, [])
print(merges.items())
tokenize("প্রেমিকের সঙ্গে বান্ধবীর ঘনিষ্ঠতার ভিডিয়ো তুলে টাকা আদায়!")

dict_items([(('্', 'র'), '্র'), (('ত', 'া'), 'তা'), (('ক', 'া'), 'কা'), (('ে', 'র'), 'ের'), (('্র', 'ে'), '্রে'), (('তা', 'র'), 'তার'), (('ু', 'ল'), 'ুল'), (('ি', 'ক'), 'িক'), (('া', 'ন'), 'ান'), (('এ', 'ক'), 'এক'), (('ছ', 'ে'), 'ছে'), (('্', 'ক'), '্ক'), (('প', '্রে'), 'প্রে'), (('প্রে', 'ম'), 'প্রেম'), (('স', 'ঙ'), 'সঙ'), (('সঙ', '্'), 'সঙ্'), (('সঙ্', 'গ'), 'সঙ্গ'), (('সঙ্গ', 'ে'), 'সঙ্গে'), (('ুল', 'ে'), 'ুলে'), (('া', 'য়'), 'ায়'), (('ক', 'ল'), 'কল'), (('কল', 'কা'), 'কলকা'), (('শ', 'িক'), 'শিক'), (('শিক', '্'), 'শিক্'), (('শিক্', 'ষ'), 'শিক্ষ'), (('শিক্ষ', 'ি'), 'শিক্ষি'), (('শিক্ষি', 'কা'), 'শিক্ষিকা'), (('গ', '্রে'), 'গ্রে'), (('গ্রে', 'ফ'), 'গ্রেফ'), (('গ্রেফ', 'তার'), 'গ্রেফতার'), (('দ', 'ু'), 'দু'), (('এক', 'ট'), 'একট'), (('একট', 'ি'), 'একটি'), (('স', '্ক'), 'স্ক'), (('য়', 'ে'), 'য়ে'), (('য়ে', 'ছে'), 'য়েছে'), (('ক', 'র'), 'কর'), (('প্রেম', 'িক'), 'প্রেমিক'), (('প্রেমিক', 'ের'), 'প্রেমিকের'), (('ব', 'ান'), 'বান'), (('বান', '্'), 'বান্'), (('বান্', 'ধ'), 'বান্ধ'), (('বান্ধ', 'ব'

['প',
 '্',
 'র',
 'ে',
 'ম',
 'ি',
 'ক',
 'স',
 'ঙ',
 'গ',
 'ব',
 'া',
 'ন',
 'ধ',
 'ী',
 'ঘ',
 'ষ',
 'ঠ',
 'ত',
 'ভ',
 'ড',
 'য়',
 'ো',
 'ু',
 'ল',
 'ট',
 'আ',
 'দ',
 '!']

In [19]:
tokenize("প্রেমিকের সঙ্গে বান্ধবীর ঘনিষ্ঠতার ভিডিয়ো তুলে টাকা আদায়!")

['প',
 '্',
 'র',
 'ে',
 'ম',
 'ি',
 'ক',
 'স',
 'ঙ',
 'গ',
 'ব',
 'া',
 'ন',
 'ধ',
 'ী',
 'ঘ',
 'ষ',
 'ঠ',
 'ত',
 'ভ',
 'ড',
 'য়',
 'ো',
 'ু',
 'ল',
 'ট',
 'আ',
 'দ',
 '!']